In [274]:

# sourcery skip: for-index-underscore
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import json

In [275]:
url = "https://www.tmz.com"
req = requests.get(url)
doc = BeautifulSoup(req.text)

In [276]:
raw_titles = doc.select("header > a > h2")

In [277]:
articles = doc.select("article > a")
articles[0]['href']

'https://www.tmz.com/2022/08/03/lala-kent-50-cent-visit-set-feud-vanderpump-rules/'

In [278]:
# Grabs all the headlines, appends period if doesn't end in !
count = 0
headlines = []
valid_punct = ["'", "!", "?"]
for title in enumerate(raw_titles):
    print("- - -")
    headline = raw_titles[count].get_text().strip().replace("\n", " ") + "."
    if headline[-2] in valid_punct:
        headline = headline[:-1]
    else: pass

    print(headline)
    headlines.append(headline)
    count += 1

- - -
Rep. Nancy Pelosi Little Retail Therapy in Hawaii Before Big Taiwan Trip.
- - -
L.A. Dodgers Fans Mourn Vin Scully's Death At Stadium ... Build Makeshift Memorial.
- - -
Crash Landing Mystery 911 Recording After Pilot Jumped Out of Plane ... Minutes Before Emergency Landing.
- - -
Beyonce Monica Lewinsky Gets Stung ... For Asking Bey to Change 'Partition' Lyric.
- - -
Wendy Williams Not Married to NYPD Cop ... Despite Report.
- - -
Tom Brady Gisele Wishes Him Happy 45th Bday!! ... 'You Are So Loved'
- - -
Jeff Bezos Mega Yacht Towed Away in Early AM ... Amid Dutch Bridge Outrage.
- - -
Post Roe v. Wade Kansas Votes Overwhelmingly for Abortion Rights.
- - -
Jon Stewart Vet Bill Pressure Works ... GOP Senators Flip, Pass PACT Act.
- - -
Vin Scully Dodgers Broadcasting Legend ... Dead At 94.
- - -
YING YANG TWINS' D-Roc I'm All Good After Collapse On Stage ... Life Changes On Deck.
- - -
'Vanderpump Rules' Lala Kent Hangin' with 50 Cent After '19 Feud ... We All Good?
- - -
Hasim Ra

In [285]:
def get_article_href(article, position):
    articles = doc.select("article > a")
    href = articles[position]['href']
    if href:
        return href
    else: pass

In [286]:
def get_tags(article, position):
    tags = doc.select("section.tag-cloud > ul > a")
    tag_list = []
    for tag in tags:
        data = json.loads(tag['data-context'])
        if data['pos'] == position:
            tag_list.append(tag.text.strip())
    return tag_list

In [287]:
def break_timestamp(article, position):
    timestamps = doc.select(".article")
        # grab, clean timestamp
    timestamp = timestamps[position].text.split("PT")[-20:]
    timestamp = timestamp[0][-20:]
    timestamp = timestamp.strip()
    # timestamp = datetime.st
    calendar = timestamp[:9]
    clock =  timestamp[-8:].strip()
    
    date_a_frame = pd.DataFrame()
    # date_a_frame["calendar"] = datetime.strptime(calendar, "%b %d %Y %I:%M%p")
    # date_a_frame["clock"] = clock
    year = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").year
    month = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").month
    day = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").day
    hour = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").hour
    minute = datetime.strptime(timestamp, "%m/%d/%Y %I:%M %p").minute

    datetimes = {
        "calendar": calendar,
        "clock": clock,
        "year": year,
        "month": month,
        "day": day,
        "hour": hour,
        "minute": minute
    }
    meta = []
    meta.append(datetimes)
    return datetimes

In [288]:
break_timestamp(raw_titles, 3)

{'calendar': '8/3/2022 ',
 'clock': '7:05 AM',
 'year': 2022,
 'month': 8,
 'day': 3,
 'hour': 7,
 'minute': 5}

In [290]:
# init dataframe of tags
tags = doc.select("section.tag-cloud > ul > a")
articles = doc.select("article > a")

valid_punct = ["'", "!", "?"]

dicts = []

count = 0
for article in articles:
    url = get_article_href(article, count)
for article_position, article in enumerate(raw_titles, start=1):
    headline = raw_titles[count].get_text().strip().replace("\n", " ") + "."
    if headline[-2] in valid_punct:
        headline = headline[:-1]
    tag_list = get_tags(article, count + 1)

    datetimes = break_timestamp(article, count)
    timestamps = doc.select(".article")
    # grab, clean timestamp
    timestamp = timestamps[count].text.split("PT")[-20:]
    timestamp = timestamp[0][-20:]
    timestamp = timestamp.strip()
    
    
    url = get_article_href(article, count)

    article_dict = {
        "headline": headline,
        "tags": tag_list,
        "timestamp": timestamp,
        "url": url,
        "year": datetimes["year"],
        "month": datetimes["month"],
        "day": datetimes["day"],
        "hour": datetimes["hour"],
        "minute": datetimes["minute"],
    }
    dicts.append(article_dict)
    count += 1
current = pd.DataFrame(dicts)
current


,headline,tags,timestamp,url,year,month,day,hour,minute
0,Rep. Nancy Pelosi Little Retail Therapy in Haw...,"[Nancy Pelosi, Fashion, Politix, Exclusive, Co...",8/3/2022 1:00 AM,https://www.tmz.com/2022/08/03/lala-kent-50-ce...,2022,8,3,1,0
1,L.A. Dodgers Fans Mourn Vin Scully's Death At ...,"[Vin Scully, Los Angeles Dodgers, RIP, Celebri...",8/3/2022 6:13 AM,https://www.tmz.com/photos/2022/08/03/billie-e...,2022,8,3,6,13
2,Crash Landing Mystery 911 Recording After Pilo...,[],8/3/2022 7:28 AM,https://www.tmz.com/2022/08/03/ying-yang-twins...,2022,8,3,7,28
3,Beyonce Monica Lewinsky Gets Stung ... For Ask...,"[Monica Lewinsky, Beyonce Knowles, Bill Clinto...",8/3/2022 7:05 AM,https://www.tmz.com/photos/2022/08/03/martha-s...,2022,8,3,7,5
4,Wendy Williams Not Married to NYPD Cop ... Des...,"[Wendy Williams, TV, Relationships, Tying The ...",8/3/2022 6:52 AM,https://www.tmz.com/2022/08/02/dodgers-vin-scu...,2022,8,3,6,52
5,Tom Brady Gisele Wishes Him Happy 45th Bday!! ...,"[Tom Brady, Gisele Bundchen, TMZ Sports, Coupl...",8/3/2022 6:52 AM,https://www.tmz.com/2022/06/23/nancy-pelosi-hu...,2022,8,3,6,52
6,Jeff Bezos Mega Yacht Towed Away in Early AM ....,"[Jeff Bezos, Money, Controversial S#!T]",8/3/2022 7:10 AM,https://www.tmz.com/2022/05/21/nancy-pelosi-ba...,2022,8,3,7,10
7,Post Roe v. Wade Kansas Votes Overwhelmingly f...,"[White House, Politix, Controversial S#!T]",8/3/2022 6:14 AM,https://www.tmz.com/2021/01/04/vin-scully-wife...,2022,8,3,6,14
8,Jon Stewart Vet Bill Pressure Works ... GOP Se...,"[Jon Stewart, Ted Cruz, You Might Want to Reth...",8/2/2022 6:22 PM,https://www.tmz.com/2019/09/11/vin-scully-firs...,2022,8,2,18,22
9,Vin Scully Dodgers Broadcasting Legend ... Dea...,"[Vin Scully, Los Angeles Dodgers, RIP, Celebri...",8/2/2022 8:30 PM,https://www.tmz.com/2022/08/01/beyonce-replace...,2022,8,2,20,30


In [261]:

# This is reading the headlines.csv file and converting it to a dataframe.
existing = pd.read_csv("../headlines.csv")
existing = existing.dropna(axis=1).set_index("Unnamed: 0")
if 'tags' not in existing:
    existing["tags"] = None
existing

,headline,timestamp,year,month,day,hour,minute,tags
Unnamed: 0,,,,,,,,
0,Vin Scully Dodgers Broadcasting Legend ... De...,8/2/2022 8:30 PM,2022,8,2,20,30,None
1,Jon Stewart Vet Bill Pressure Works ... GOP S...,8/2/2022 6:22 PM,2022,8,2,18,22,None
2,Willow Smith Stops Show After Fan Faints in C...,8/2/2022 6:12 PM,2022,8,2,18,12,None
3,Killer Mike ATL's Black Biz Owners in Danger ...,8/2/2022 2:57 PM,2022,8,2,14,57,None
4,'Ferris Bueller' Star Edie McClurg Allegedly ...,8/2/2022 1:41 PM,2022,8,2,13,41,None
...,...,...,...,...,...,...,...,...
154,Taco Bell Employee Allegedly Throws Boiling H...,7/26/2022 6:27 AM,2022,7,26,6,27,None
155,Brittney Griner Sends Message To Wife In Cour...,7/26/2022 6:13 AM,2022,7,26,6,13,None
156,PetSmart Sued Your Groomers Killed My Dog!!!,7/26/2022 1:00 AM,2022,7,26,1,0,None


In [264]:
dfs = [current, existing]
existing = existing[["headline", "tags", "timestamp", "year", "month", "day", "hour", "minute"]]
export = pd.concat([current, existing])

export = export.sort_values(
    by=["year", "month", "day", "hour", "minute"],
    ascending=[False, False, False, False, False],
    ignore_index=True,
)
export = export.drop_duplicates(subset=["timestamp"], keep="first", ignore_index=True)
export

,headline,tags,timestamp,year,month,day,hour,minute
0,Vin Scully Dodgers Broadcasting Legend ... Dea...,"[Vin Scully, TMZ Sports]",8/2/2022 8:30 PM,2022,8,2,20,30
1,Jon Stewart Vet Bill Pressure Works ... GOP Se...,"[Jon Stewart, Ted Cruz, You Might Want to Reth...",8/2/2022 6:22 PM,2022,8,2,18,22
2,Willow Smith Stops Show After Fan Faints in Cr...,"[Willow Smith, Machine Gun Kelly, Nurse!, Cont...",8/2/2022 6:12 PM,2022,8,2,18,12
3,LeBron James Hits Lakers Facility W/ Sons ... ...,"[LeBron James, Lebron James jr, Bryce James, T...",8/2/2022 2:57 PM,2022,8,2,14,57
4,'Ferris Bueller' Star Edie McClurg Allegedly E...,"[Edie McClurg, Movies, Sex, Nurse!, Exclusive,...",8/2/2022 1:41 PM,2022,8,2,13,41
...,...,...,...,...,...,...,...,...
154,Taco Bell Employee Allegedly Throws Boiling H...,None,7/26/2022 6:27 AM,2022,7,26,6,27
155,Brittney Griner Sends Message To Wife In Cour...,None,7/26/2022 6:13 AM,2022,7,26,6,13
156,PetSmart Sued Your Groomers Killed My Dog!!!,None,7/26/2022 1:00 AM,2022,7,26,1,0
157,'RHOA' STAR SHEREE WHITFIELD Dating Martell H...,None,7/26/2022 12:50 AM,2022,7,26,0,50


In [252]:
# Writing the dataframe to a csv file.
export.to_csv("headlines.csv")
export.to_json("headlines.json")
